In [119]:
import zipfile
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np



    Розарпаковка архіву. Переведення даних з файлів .csv у словник датасетів. 

In [117]:
zip_file_path = 'homework.zip'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall()

data_folder = 'data'

subdirectories = [subdir for subdir in os.listdir(data_folder) if os.path.isdir(os.path.join(data_folder, subdir))]
print(subdirectories)
dataframes = {}

for subdir in subdirectories:
    subdir_path = os.path.join(data_folder, subdir)
    files = os.listdir(subdir_path)

    df = pd.DataFrame()

    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(subdir_path, file)
            data = pd.read_csv(file_path)
            df = pd.concat([df, data], ignore_index=True)

    dataframes[subdir] = df

for subdir, df in dataframes.items():
    print(f"Dataframe для {subdir}:")
    print(df.head())

['running', 'idle', 'stairs', 'walking']
Dataframe для running:
   accelerometer_X  accelerometer_Y  accelerometer_Z
0        -2.983175        10.347740         1.575385
1        -4.070143        -5.482722        -6.818686
2         0.081403         9.404425        -3.203442
3         3.806781         7.503428         1.221043
4        -9.988609        -5.995081        -5.080496
Dataframe для idle:
   accelerometer_X  accelerometer_Y  accelerometer_Z
0         0.186748        -0.043096         9.744401
1         0.325611         0.023942         9.773131
2         0.215478        -0.023942         9.725247
3         0.272939        -0.110133         9.777920
4         0.263362        -0.105345         9.801862
Dataframe для stairs:
   accelerometer_X  accelerometer_Y  accelerometer_Z
0         5.923255       -18.195932        10.486604
1         2.595315        -7.762002       -10.606314
2        -6.287173       -20.609290       -38.086960
3         1.498770       -14.719552        -5.

    Додавання до кожного датасету стовпця, що відповідає дії: 0 - 'running', 1 - 'idle', 2 - 'stairs', 3 - 'walking'.
    Обєднання всіх датасетів у один датафрейм

In [120]:
scaler = StandardScaler()


for index, (subdir, df) in enumerate(dataframes.items()):
    df['Action'] = index
    dataframes[subdir] = df

merged_df = pd.concat(dataframes.values(), ignore_index=True)
print(merged_df)

        accelerometer_X  accelerometer_Y  accelerometer_Z  Action
0             -2.983175        10.347740         1.575385       0
1             -4.070143        -5.482722        -6.818686       0
2              0.081403         9.404425        -3.203442       0
3              3.806781         7.503428         1.221043       0
4             -9.988609        -5.995081        -5.080496       0
...                 ...              ...              ...     ...
193855        -2.365471       -13.239936        -1.675941       3
193856        -2.820370        -3.457227        -0.967257       3
193857        -1.192312        -8.413225         0.512359       3
193858        -2.820370        -2.643199        -0.478840       3
193859        -0.316035        -8.657434        -0.531513       3

[193860 rows x 4 columns]


    Нормалізація за допомогою StandardScaler

In [121]:

scaler = StandardScaler()

# Нормалізуємо дані, крім стовпця "Action"
normalized_data = merged_df.drop(columns=['Action'])
normalized_data = scaler.fit_transform(normalized_data)

# Створюємо DataFrame з нормалізованими даними
normalized_df = pd.DataFrame(normalized_data, columns=merged_df.columns[:-1])  # без стовпця "Action"

# Додаємо стовпець "Action" зі списку
normalized_df['Action'] = merged_df['Action']

print(normalized_df)


        accelerometer_X  accelerometer_Y  accelerometer_Z  Action
0             -0.583797         0.701410        -0.031914       0
1             -0.713123        -0.567666        -1.199123       0
2             -0.219177         0.625788        -0.696418       0
3              0.224065         0.473391        -0.081186       0
4             -1.417297        -0.608740        -0.957425       0
...                 ...              ...              ...     ...
193855        -0.510303        -1.189536        -0.484016       3
193856        -0.564427        -0.405288        -0.385472       3
193857        -0.370722        -0.802594        -0.179729       3
193858        -0.564427        -0.340030        -0.317557       3
193859        -0.266464        -0.822172        -0.324882       3

[193860 rows x 4 columns]


    Визначення списку індексів для початкових індексів кожного сегмента.
    Поділ даних на сегменти та розрахнок часових ознак для кожного сегмента.
    Створенння нового датафрейму, який містить часові ознаки для кожного сегмента разом з відповідними мітками класів.

In [122]:
# Крок переміщення між сегментами (50% перекриття)
segment_size = 208
step_size = int(segment_size / 2)

# Створення списку індексів для початкових індексів кожного сегмента
start_indexes = list(range(0, len(normalized_df), step_size))

In [136]:
# Список для збереження часових ознак для кожного сегмента
time_domain_features = []

# Проходимося по кожному початковому індексу кожного сегмента
for start_index in start_indexes:
    # Визначаємо кінець поточного сегмента
    end_index = start_index + segment_size
    # Вибираємо дані для поточного сегмента
    segment_data = normalized_df.iloc[start_index:end_index, :-1]  # без стовпця "Action"

    # Розраховуємо часові ознаки для поточного сегмента
    segment_features = {
        'mean_x': segment_data['accelerometer_X'].mean(),
        'mean_y': segment_data['accelerometer_Y'].mean(),
        'mean_z': segment_data['accelerometer_Z'].mean(),
        'std_x': segment_data['accelerometer_X'].std(),
        'std_y': segment_data['accelerometer_Y'].std(),
        'std_z': segment_data['accelerometer_Z'].std(),
        'min_x': segment_data['accelerometer_X'].min(),
        'min_y': segment_data['accelerometer_Y'].min(),
        'min_z': segment_data['accelerometer_Z'].min(),
        'max_x': segment_data['accelerometer_X'].max(),
        'max_y': segment_data['accelerometer_Y'].max(),
        'max_z': segment_data['accelerometer_Z'].max(),
        'median_x': segment_data['accelerometer_X'].median(),
        'median_y': segment_data['accelerometer_Y'].median(),
        'median_z': segment_data['accelerometer_Z'].median(),
        '25th_percentile_x': segment_data['accelerometer_X'].quantile(0.25),
        '25th_percentile_y': segment_data['accelerometer_Y'].quantile(0.25),
        '25th_percentile_z': segment_data['accelerometer_Z'].quantile(0.25),
        '75th_percentile_x': segment_data['accelerometer_X'].quantile(0.75),
        '75th_percentile_y': segment_data['accelerometer_Y'].quantile(0.75),
        '75th_percentile_z': segment_data['accelerometer_Z'].quantile(0.75),
        'rms_x': np.sqrt(np.mean(segment_data['accelerometer_X'] ** 2)),
        'rms_y': np.sqrt(np.mean(segment_data['accelerometer_Y'] ** 2)),
        'rms_z': np.sqrt(np.mean(segment_data['accelerometer_Z'] ** 2))
    }

    # Додаємо часові ознаки до списку
    time_domain_features.append(segment_features)

# Створюємо DataFrame з часовими ознаками
time_domain_df = pd.DataFrame(time_domain_features)

# Додаємо стовпець "Action" зі списку індексів
time_domain_df['Action'] = normalized_df['Action'].iloc[::step_size].values

print(time_domain_df.head())



     mean_x    mean_y    mean_z     std_x     std_y     std_z     min_x  \
0  0.258703  0.452324  0.027589  1.116132  0.953011  1.083355 -3.138419   
1  0.551210  0.503590 -0.010316  1.103953  0.962263  1.078645 -1.354627   
2  0.611951  0.614713 -0.057033  1.135218  0.958601  0.955942 -1.354627   
3  0.591928  0.582062 -0.048678  1.130616  0.997452  0.987720 -1.066919   
4  0.549523  0.559884 -0.071938  1.191009  1.085022  0.998333 -1.458886   

      min_y     min_z     max_x  ...  25th_percentile_x  25th_percentile_y  \
0 -1.488955 -2.576071  4.257108  ...          -0.432252          -0.245790   
1 -1.595287 -2.032084  4.327184  ...          -0.226868          -0.203276   
2 -1.595287 -2.032084  4.433721  ...          -0.280422           0.196908   
3 -2.922327 -2.869039  4.433721  ...          -0.194679           0.048543   
4 -2.922327 -2.869039  4.433721  ...          -0.341666          -0.285809   

   25th_percentile_z  75th_percentile_x  75th_percentile_y  75th_percentile_z  \

    Розділення датасету на навчальний і тестовий.

In [137]:
from sklearn.model_selection import train_test_split

X = time_domain_df.drop(columns=['Action'])  # Ознаки
y = time_domain_df['Action']  # Цільова змінна

# Розділення даних у співвідношенні 80% для навчання та 20% для тестування
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Розмірність навчального датасету:", X_train.shape, y_train.shape)
print("Розмірність тестового датасету:", X_test.shape, y_test.shape)


Розмірність навчального датасету: (1492, 24) (1492,)
Розмірність тестового датасету: (373, 24) (373,)


In [138]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Ініціалізуємо класифікатор SVM
svm_classifier = SVC()

# Навчаємо модель SVM на навчальних даних
svm_classifier.fit(X_train, y_train)

# Передбачення за допомогою моделі SVM на тестових даних
svm_predictions = svm_classifier.predict(X_test)

# Оцінка точності моделі SVM
svm_accuracy = accuracy_score(y_test, svm_predictions)
print("Точність моделі SVM:", svm_accuracy)

# Ініціалізуємо класифікатор випадкового лісу
rf_classifier = RandomForestClassifier()

# Навчаємо модель випадкового лісу на навчальних даних
rf_classifier.fit(X_train, y_train)

# Передбачення за допомогою моделі випадкового лісу на тестових даних
rf_predictions = rf_classifier.predict(X_test)

# Оцінка точності моделі випадкового лісу
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Точність моделі випадкового лісу:", rf_accuracy)


Точність моделі SVM: 1.0
Точність моделі випадкового лісу: 1.0


In [139]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Навчання моделі SVM
svm_model = SVC()
svm_model.fit(X_train, y_train)

# Передбачення класів для тестового датасету з використанням моделі SVM
svm_predictions = svm_model.predict(X_test)

# Навчання моделі випадкового лісу
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

# Передбачення класів для тестового датасету з використанням моделі випадкового лісу
rf_predictions = rf_model.predict(X_test)

# Виведення classification report для моделі SVM
print("Classification Report для моделі SVM:")
print(classification_report(y_test, svm_predictions))

# Виведення classification report для моделі випадкового лісу
print("Classification Report для моделі випадкового лісу:")
print(classification_report(y_test, rf_predictions))


Classification Report для моделі SVM:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       205
           1       1.00      1.00      1.00        54
           2       1.00      1.00      1.00        13
           3       1.00      1.00      1.00       101

    accuracy                           1.00       373
   macro avg       1.00      1.00      1.00       373
weighted avg       1.00      1.00      1.00       373

Classification Report для моделі випадкового лісу:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       205
           1       1.00      1.00      1.00        54
           2       1.00      1.00      1.00        13
           3       1.00      1.00      1.00       101

    accuracy                           1.00       373
   macro avg       1.00      1.00      1.00       373
weighted avg       1.00      1.00      1.00       373



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Виберемо підмножину ознак для візуалізації (наприклад, перші три ознаки)
features_subset = time_domain_df.iloc[:, :3]

# Додамо до набору ознак стовпець "Action"
features_subset['Action'] = time_domain_df['Action']

# Використаємо бібліотеку seaborn для побудови графіків розсіювання
sns.pairplot(features_subset, hue='Action')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Виберемо підмножину ознак для візуалізації (наприклад, перші три ознаки)
features_subset = time_domain_df.iloc[:, 3:6]

# Додамо до набору ознак стовпець "Action"
features_subset['Action'] = time_domain_df['Action']

# Використаємо бібліотеку seaborn для побудови графіків розсіювання
sns.pairplot(features_subset, hue='Action')
plt.show()


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# Виберемо перші три ознаки для візуалізації
features_subset = time_domain_df.iloc[:, 2:6]

# Додамо до набору ознак стовпець "Action"
features_subset['Action'] = time_domain_df['Action']

# Розділимо дані за класами
class_labels = features_subset['Action'].unique()

# Створимо графік
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Пройдемося по кожному класу і відобразимо його дані на графіку
for label in class_labels:
    class_data = features_subset[features_subset['Action'] == label]
    ax.scatter(class_data.iloc[:, 0], class_data.iloc[:, 1], class_data.iloc[:, 2], label=f'Action {label}')

ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.set_zlabel('Feature 3')
ax.set_title('3D Scatter Plot of Features')
ax.legend()
plt.show()


In [ ]:
# Кількість кластерів (рівна кількості класів)
n_clusters = len(time_domain_df['Action'].unique())

# Застосуємо k-means для кожного класу та знайдемо центроїди
centroids = []
for label in class_labels:
    class_data = time_domain_df[time_domain_df['Action'] == label].iloc[:, :3]  # Виберемо перші три ознаки
    kmeans = KMeans(n_clusters=1, random_state=42)
    kmeans.fit(class_data)
    centroids.append(kmeans.cluster_centers_[0])

# Створимо графік
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Пройдемося по кожному класу і відобразимо його дані та центроїд на графіку
for label in class_labels:
    class_data = time_domain_df[time_domain_df['Action'] == label].iloc[:, :3]  # Виберемо перші три ознаки
    ax.scatter(class_data.iloc[:, 0], class_data.iloc[:, 1], class_data.iloc[:, 2], label=f'Action {label}')

# Додамо центроїди на графік
centroids = np.array(centroids)
ax.scatter(centroids[:, 0], centroids[:, 1], centroids[:, 2], c='red', marker='x', s=200, label='Centroids')

ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.set_zlabel('Feature 3')
ax.set_title('3D Scatter Plot of Features with Centroids')
ax.legend()
plt.show()
